In [1]:
from pathlib import Path
from validphys.loader import _get_nnpdf_profile
from validphys.api import API
import numpy as np
import pandas as pd

profile = _get_nnpdf_profile()
yaml_db = Path(profile["data_path"]) / "yamldb"

The `yaml_db` folder is a temporary thing as it contains files that look like:

```yaml
conversion_factor: 1.0
operands:
- - NMC_NC_EM_D_F2
- - NMC_NC_EM_P_F2
operation: RATIO
target_dataset: NMCPD
```

This information will eventually be part of the new commondata format of course.

The `operation` is applied to the first level of the list while the second level is just concatenated. This is necessary since `pineappl` fktables might contain one layer of concatenation which is already done for the "classic" fktables.

The `pineappl` fktables will live inside the appropiate `theory_xxx` folder `/pineappls`.

In [2]:
target_ds = "ATLASDY2D8TEV"
cfac = []
old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac + ["oldmode"]}, theoryid=200, use_cuts="internal")
ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")

In [3]:
# Let's try to get a prediction out of it
from validphys.convolution import central_predictions
pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
new_cp = central_predictions(ds, pdf)
cp = central_predictions(old_ds, pdf)
pd.concat([new_cp, cp, new_cp/cp], axis=1, keys=["vp", "pine", "ratio"])


-- Reading COMMONDATA for Dataset: ATLASDY2D8TEV
nData: 48 nSys: 37
-- COMMONDATA Files for ATLASDY2D8TEV successfully read.

LHAPDF 6.4.0 loading /usr/share/lhapdf/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1


,vp,pine,ratio
,0,0,0
data,,,
0,2.777670,2.780341,0.999040
1,2.771371,2.773974,0.999062
2,2.758831,2.761445,0.999053
3,2.738773,2.741277,0.999087
4,2.708849,2.710993,0.999209
5,2.685853,2.687905,0.999237
6,2.603562,2.605157,0.999388
7,2.372306,2.373824,0.999360


In [4]:
pine_fkspec = ds.fkspecs[0]
old_fkspec = old_ds.fkspecs[0]

In [5]:
import pineappl
pines = [pineappl.fk_table.FkTable.read(i.as_posix()) for i in pine_fkspec.fkpath]
# Inspect the pineappl prediction
res_pine = []
pp = pines[0]
lpdf = pdf.load()

for p in pines:
    res_pine.append(p.convolute_with_one(2212, lpdf.central_member.xfxQ2))
total_pine = np.concatenate(res_pine)

LHAPDF 6.4.0 loading all 101 PDFs in set NNPDF40_nnlo_as_01180
NNPDF40_nnlo_as_01180, version 1; 101 PDF members


In [6]:
# Let's inspect the content of the old fktables, remove the cfactor for now
from validphys.fkparser import load_fktable
old_fkspec.cfactors = False
old_fktabledata = load_fktable(old_fkspec)

In [7]:
print(f"hadronic?: {old_fktabledata.hadronic}")
print(f"Q: {old_fktabledata.Q0}")
print(f"n: {old_fktabledata.ndata}")
print(f"xgrid shape: {old_fktabledata.xgrid.shape}")
#old_fktabledata.sigma

hadronic?: True
Q: 1.65
n: 48
xgrid shape: (50,)


In [8]:
# First read the metadata that vp `FKTableData` needs and that all subgrids share
Q0 = np.sqrt(pp.muf2())
xgrid = pp.x_grid()
# Hadronic means in practice that not all luminosity combinations are just electron X proton
hadronic = not all(-11 in i for i in pp.lumi())
# Now prepare the concatenation of grids
fktables = []
for p in pines:
    tmp = p.table().T/p.bin_normalizations()
    fktables.append(tmp.T)
fktable = np.concatenate(fktables, axis=0)
ndata = fktable.shape[0]

In [9]:
# Now let's try to join the fktable, luminosity and xgrid into a pandas dataframe
# keeping compatibility with validphys and, hopefully, 50% of my own sanity

# Step 1), make the luminosity into a 14x14 mask for the evolution basis
eko_numbering_scheme = (22, 100, 21, 200, 203, 208, 215, 224, 235, 103, 108, 115, 124, 135)
# note that this is the same ordering that was used in fktables
flavour_map = np.zeros((14, 14), dtype=bool)
for i, j in pp.lumi():
    idx = eko_numbering_scheme.index(i)
    jdx = eko_numbering_scheme.index(j)
    flavour_map[idx,jdx] = True
    
# Step 2) prepare the indices for the dataframe
xi = np.arange(len(xgrid))
ni = np.arange(ndata)
mi = pd.MultiIndex.from_product([ni, xi, xi], names=["data", "x1", "x2"])
co = np.where(flavour_map.ravel())[0]

# Step 3) Now play with the array until we flatten it in the right way?
# The fktables for pineappl have this extra factor of x...
# The output of pineappl is (ndata, flavours, x, x)
lf = len(co)
xfktable = fktable.reshape(ndata, lf, -1)/(xgrid[:,None]*xgrid[None,:]).flatten()
fkmod = np.moveaxis(xfktable, 1, -1)
fkframe = fkmod.reshape(-1, lf)

# Uff, big
df = pd.DataFrame(fkframe, index=mi, columns=co)

from validphys.convolution import central_hadron_predictions
from validphys.coredata import FKTableData
fk = FKTableData(sigma=df, ndata=ndata,  Q0=Q0, metadata=None, hadronic=True, xgrid=xgrid)
central_hadron_predictions(fk, pdf)

,0
data,
0,2.777670
1,2.771371
2,2.758831
3,2.738773
4,2.708849
5,2.685853
6,2.603562
7,2.372306
8,1.971403


In [10]:
# Create a luminosity tensor and check that the results are correct
from validphys.pdfbases import evolution

evol_basis = (
    "photon",
    "singlet",
    "g",
    "V",
    "V3",
    "V8",
    "V15",
    "V24",
    "V35",
    "T3",
    "T8",
    "T15",
    "T24",
    "T35",
)
total_pdf = evolution.grid_values(pdf, evol_basis, xgrid, [Q0]).squeeze()[0]/xgrid
print(total_pdf.shape)
lumi = np.einsum('ij,kl->ikjl', total_pdf, total_pdf)
lumi_masked = lumi[flavour_map]
print(fktable.shape)
print(lumi_masked.shape)
res = np.einsum('ijkl,jkl->i', fktable, lumi_masked)
#pd.concat([pd.DataFrame(res), cp, pd.DataFrame(res)/cp,  ], axis=1)

(14, 50)
(48, 91, 50, 50)
(91, 50, 50)


In [11]:
xfktable.reshape(48,91,-1).shape

(48, 91, 2500)

In [12]:
from validphys.fkparser import open_fkpath, _parse_string, _parse_header, _build_sigma
from validphys.fkparser import _parse_flavour_map, _parse_hadronic_fast_kernel
try:
    f.close()
except:
    pass
f = open_fkpath(old_fkspec.fkpath)
line_and_stream = enumerate(f, start=1)
lineno, header = next(line_and_stream)
res = {}
while True:
    marker, header_name = _parse_header(lineno, header)
    if header_name == "FastKernel":
        break
    if header_name == "FlavourMap":
        out, lineno, header = _parse_flavour_map(line_and_stream)
    else:
        out, lineno, header = _parse_string(line_and_stream)
    res[header_name] = out   

In [13]:
res["FlavourMap"].shape

(14, 14)

In [14]:
i_hate_pandas = _parse_hadronic_fast_kernel(f)

In [15]:
i_hate_pandas

0    1    2    3    4    5    6    7    8    9    ...  186  187  \
data x1 x2                                                    ...             
0    8  9     0    0    0    0    0    0    0    0    0    0  ...    0    0   
        10    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        11    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        12    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        13    0    0    0    0    0    0    0    0    0    0  ...    0    0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
47   49 45    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        46    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        47    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        48    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        49    0    0    0    0    0    0    0    0    0    0  ...    0    0   

            188  189  190  191  192  193  194  195  
data x1 x2                                          
0    8  9     0    0    0    0    0    0    0    0  
        10    0    0    0    0    0    0    0    0  
        11    0    0    0    0    0    0    0    0  
        12    0    0    0    0    0    0    0    0  
        13    0    0    0    0    0    0    0    0  
...         ...  ...  ...  ...  ...  ...  ...  ...  
47   49 45    0    0    0    0    0    0    0    0  
        46    0    0    0    0    0    0    0    0  
        47    0    0    0    0    0    0    0    0  
        48    0    0    0    0    0    0    0    0  
        49    0    0    0    0    0    0    0    0  

[86050 rows x 196 columns]

In [16]:
old_fktabledata.sigma

15            16            17            18   \
data x1 x2                                                           
0    8  9   3.421595e-10  5.264124e-12 -3.880182e-11 -1.017778e-11   
        10  1.175460e-09  4.165056e-11 -1.502275e-10 -5.499317e-11   
        11 -1.687773e-08 -3.699979e-10  1.090329e-09 -4.637641e-10   
        12 -2.612924e-07 -3.892605e-09 -1.174226e-08 -1.277032e-08   
        13 -7.590526e-07 -2.464289e-08 -4.152872e-08 -3.874544e-08   
...                  ...           ...           ...           ...   
47   49 45  2.821460e-06  2.549644e-06 -5.553106e-08 -3.051969e-08   
        46  2.765775e-06  2.525223e-06 -6.399612e-08 -3.847599e-08   
        47  2.687240e-06  2.501610e-06 -4.170865e-08 -4.156584e-08   
        48  2.621881e-06  2.479730e-06 -2.514065e-08 -5.231382e-08   
        49  2.566434e-06  2.456288e-06 -2.582100e-08 -7.333546e-08   

                     19            20            23            24   \
data x1 x2                                                           
0    8  9  -3.392594e-12  3.392596e-12 -3.594754e-11 -1.198252e-11   
        10 -1.833106e-11  1.833106e-11 -1.421209e-10 -4.737362e-11   
        11 -1.545879e-10  1.545879e-10  8.680430e-10  2.893476e-10   
        12 -4.256774e-09  4.256774e-09  6.963248e-09  2.321084e-09   
        13 -1.291514e-08  1.291515e-08  1.899462e-08  6.331540e-09   
...                  ...           ...           ...           ...   
47   49 45 -1.017323e-08  1.017323e-08  4.265377e-07  1.421792e-07   
        46 -1.282533e-08  1.282533e-08  4.224528e-07  1.408176e-07   
        47 -1.385528e-08  1.385528e-08  4.256720e-07  1.418907e-07   
        48 -1.743794e-08  1.743794e-08  4.217841e-07  1.405947e-07   
        49 -2.444515e-08  2.444515e-08  4.063643e-07  1.354548e-07   

                     25            29   ...           151           155  \
data x1 x2                              ...                               
0    8  9   1.198252e-11  5.407421e-12  ... -3.991088e-12  1.198351e-11   
        10  4.737361e-11  1.858761e-11  ... -1.577881e-11  4.743022e-11   
        11 -2.893475e-10 -2.664041e-10  ...  9.638463e-11 -2.894811e-10   
        12 -2.321083e-09 -4.075159e-09  ...  7.727908e-10 -2.320180e-09   
        13 -6.331544e-09 -1.182504e-08  ...  2.107874e-09 -6.327810e-09   
...                  ...           ...  ...           ...           ...   
47   49 45 -1.421792e-07  2.923133e-06  ...  1.762537e-09 -1.371116e-07   
        46 -1.408176e-07  2.859848e-06  ...  1.689228e-09 -1.345498e-07   
        47 -1.418907e-07  2.753591e-06  ...  1.730347e-09 -1.324512e-07   
        48 -1.405947e-07  2.669326e-06  ...  1.714869e-09 -1.303148e-07   
        49 -1.354548e-07  2.604723e-06  ...  1.592858e-09 -1.277852e-07   

                     156           157           158           159  \
data x1 x2                                                           
0    8  9   1.919344e-13  3.389330e-12 -3.389407e-12 -1.129802e-12   
        10  1.702693e-12  1.831406e-11 -1.831501e-11 -6.105004e-12   
        11 -2.896136e-12  1.544926e-10 -1.544999e-10 -5.149993e-11   
        12 -4.017064e-11  4.251713e-09 -4.251828e-09 -1.417276e-09   
        13 -2.069524e-10  1.289884e-08 -1.289977e-08 -4.299924e-09   
...                  ...           ...           ...           ...   
47   49 45 -2.438241e-07  1.410483e-10 -1.416208e-10 -4.720695e-11   
        46 -2.394566e-07  1.857575e-10 -1.865098e-10 -6.216994e-11   
        47 -2.352476e-07  3.490110e-10 -3.499210e-10 -1.166403e-10   
        48 -2.313411e-07  4.186209e-10 -4.196703e-10 -1.398901e-10   
        49 -2.274307e-07  3.487613e-10 -3.499316e-10 -1.166438e-10   

                     160           163           164           165  
data x1 x2                                                          
0    8  9  -1.518243e-11 -1.197326e-11 -3.991088e-12  1.058912e-10  
        10 -6.224530e-11 -4.733643e-11 -1.577881e-11  3.592991e-10  
        11  2.601000e-10  2.891540e-